In [ ]:
%run markethistory_esi.py

In [1]:
import datetime
import pandas as pd
import graphlab as gl
import graphlab.aggregate as agg
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format

days_back = lambda n: datetime.date.today() - datetime.timedelta(days=n)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1486642649.log


This non-commercial license of GraphLab Create for academic use is assigned to evrial@gmail.com and will expire on August 07, 2017.


In [2]:
regions = gl.SFrame(pd.read_csv('mapRegions.csv.bz2', usecols=['regionID', 'regionName']))
empire = ['Aridia', 'Black Rise', 'The Bleak Lands', 'The Citadel', 'Derelik', 'Devoid', 'Domain', 'Essence',
          'Everyshore', 'The Forge', 'Genesis', 'Heimatar', 'Kador', 'Khanid', 'Kor-Azor', 'Lonetrek',
          'Metropolis','Molden Heath', 'Placid', 'Sinq Laison', 'Solitude', 'Tash-Murkon', 'Verge Vendor']
empire = regions.filter_by(empire, column_name='regionName')
empire

regionID,regionName
10000001,Derelik
10000002,The Forge
10000016,Lonetrek
10000020,Tash-Murkon
10000028,Molden Heath
10000030,Heimatar
10000032,Sinq Laison
10000033,The Citadel
10000036,Devoid
10000037,Everyshore


In [3]:
types = gl.SFrame(pd.read_csv("invTypes.csv.bz2",
                              usecols=['typeID', 'typeName'],
                              converters={'typeName': str}))
types

typeID,typeName
0,#System
2,Corporation
3,Region
4,Constellation
5,Solar System
6,Sun G5 (Yellow)
7,Sun K7 (Orange)
8,Sun K5 (Red Giant)
9,Sun B0 (Blue)
10,Sun F0 (White)


In [4]:
types.dtype()

[int, str]

In [5]:
prices = gl.SFrame.read_csv('history-latest.csv.gz')

Finished parsing file /Users/evrial/Dropbox/EVE markets/history-latest.csv.gz

Parsing completed. Parsed 100 lines in 1.49567 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,float,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 888559 lines. Lines per second: 347517

Read 5163925 lines. Lines per second: 611633

...

Read 13195122 lines. Lines per second: 676472

Read 17565367 lines. Lines per second: 687269

Read 22029970 lines. Lines per second: 700731

Finished parsing file /Users/evrial/Dropbox/EVE markets/history-latest.csv.gz

Parsing completed. Parsed 24667025 lines in 34.1664 secs.

In [6]:
prices['revenue'] = prices['average'] * prices['volume']
prices['date'] = prices['date'].str_to_datetime('%Y-%m-%dT%H:%M:%SZ')
prices

type_id,region_id,date,lowest,highest,average,volume,order_count,revenue
186,10000001,2016-01-04 00:00:00,34.0,34.0,34.0,10000,1,340000.0
186,10000001,2016-02-09 00:00:00,34.0,34.0,34.0,2000,1,68000.0
186,10000001,2016-02-21 00:00:00,0.01,0.01,0.01,100,1,1.0
186,10000001,2016-02-23 00:00:00,0.01,0.01,0.01,200,2,2.0
186,10000001,2016-02-28 00:00:00,0.01,0.01,0.01,100,1,1.0
186,10000001,2016-03-05 00:00:00,0.01,0.01,0.01,100,1,1.0
186,10000001,2016-03-06 00:00:00,0.01,0.01,0.01,100,1,1.0
186,10000001,2016-03-23 00:00:00,0.01,0.01,0.01,88,1,0.88
186,10000001,2016-03-24 00:00:00,0.01,0.01,0.01,200,1,2.0
186,10000001,2016-03-26 00:00:00,0.01,0.01,0.01,200,1,2.0


In [8]:
prices.show()

Canvas is accessible via web browser at the URL: http://localhost:58161/index.html
Opening Canvas in default web browser.


PLEX volume sold last 13 months:

In [7]:
# denormalized = prices.join(regions, on={'region_id': 'regionID'}).join(types, on={'type_id': 'typeID'})
prices[prices['type_id'] == 29668]['volume'].sum()

2043276

In [8]:
books = types.filter_by(['Gallente Titan', 'Caldari Titan', 'Amarr Titan', 'Minmatar Titan'], 'typeName')['typeID']

titans = prices.filter_by(books, 'type_id').join(types, {'type_id': 'typeID'}).groupby(['typeName'], agg.SUM('volume'))

print titans['Sum of volume'].sum()
titans

2158


typeName,Sum of volume
Amarr Titan,691
Minmatar Titan,451
Gallente Titan,556
Caldari Titan,460


In [12]:
days_back(1) > days_back(30)

True

In [13]:
prices[ prices['date'] > days_back(30) ] \
    .filter_by(21815, 'type_id') \
    .join(regions, on={'region_id': 'regionID'}) \
    .join(types, on={'type_id': 'typeID'})\
    .groupby(['typeName', 'regionName'],
       [agg.COUNT(),
        agg.SUM('volume'),
        agg.MIN('lowest'),
        agg.MAX('highest'),
        agg.AVG('average'),
        agg.STDV('average'),
        agg.SUM('revenue'),
        agg.QUANTILE('average',[0.25, 0.75])
       ]).topk('Sum of volume', k=15).to_dataframe()

,regionName,typeName,Count,Sum of volume,Min of lowest,Max of highest,Avg of average,Stdv of average,Sum of revenue,Quantiles of average
0,The Forge,Elite Drone AI,28,465773,"5,206,000.01","7,889,999.00","6,318,025.41","580,705.08","2,943,284,514,519.77","[5900000.0, 6900000.0]"
1,Domain,Elite Drone AI,29,24464,"5,000,253.15","9,699,883.00","5,875,915.64","716,010.29","145,308,228,293.14","[5310015.42, 6211011.64]"
2,Sinq Laison,Elite Drone AI,29,14627,"598,744.09","7,499,997.00","5,595,698.16","592,136.63","83,009,973,451.49","[5100004.01, 5999999.0]"
3,Heimatar,Elite Drone AI,29,6432,"4,367,001.01","7,400,000.00","5,495,147.17","574,404.54","36,004,132,553.09","[5000025.03, 5800000.0]"
4,Metropolis,Elite Drone AI,29,4183,"2,798,980.00","6,999,998.99","5,226,057.09","512,734.56","22,485,718,126.62","[4700000.0, 5500006.24]"
5,Cobalt Edge,Elite Drone AI,29,2854,"1,000,000.00","8,888,888.00","6,139,147.49","1,346,912.55","18,045,778,054.66","[5400000.0, 6903508.79]"
6,Lonetrek,Elite Drone AI,28,1688,"1,000,000.00","7,000,000.00","5,060,944.07","772,484.58","8,814,110,507.05","[4500100.0, 5650002.59]"
7,The Citadel,Elite Drone AI,29,1176,"4,000,000.00","7,500,000.00","4,983,309.72","848,166.81","6,237,371,197.21","[4221737.36, 5710018.38]"
8,Malpais,Elite Drone AI,23,716,"3,650,000.01","9,900,000.00","6,092,127.88","1,388,176.25","4,579,237,334.12","[5021815.32, 7350000.0]"
9,Genesis,Elite Drone AI,28,655,"4,300,001.00","8,000,000.00","4,796,076.81","508,411.15","3,368,192,492.50","[4400000.13, 5050000.38]"


WH regions market top

In [14]:
prices[(prices['region_id'] >= 11000000) & (prices['date'] > days_back(30))] \
    .join(regions, on={'region_id': 'regionID'}) \
    .join(types, on={'type_id': 'typeID'})\
    .groupby(['typeName', 'regionName'],
       [agg.COUNT(),
        agg.SUM('volume'),
        agg.AVG('revenue'),
        agg.SUM('revenue'),
        agg.STDV('revenue'),
#         agg.QUANTILE('revenue',[0.25, 0.75])
       ]).topk('Sum of revenue').to_dataframe()

,regionName,typeName,Count,Sum of volume,Avg of revenue,Sum of revenue,Stdv of revenue
0,E-R00028,Sleeper Drone AI Nexus,29,3355,"495,031,724.13","14,355,919,999.66","417,709,894.70"
1,E-R00027,Sleeper Drone AI Nexus,29,2569,"405,393,103.47","11,756,400,000.65","372,184,316.52"
2,E-R00027,Ancient Coordinates Database,29,7409,"350,655,517.20","10,169,009,998.94","208,707,773.88"
3,E-R00028,Ancient Coordinates Database,29,7760,"347,518,965.50","10,078,049,999.57","236,419,106.59"
4,E-R00027,Sleeper Data Library,29,20345,"320,751,034.68","9,301,780,005.82","177,252,740.46"
5,B-R00007,Sleeper Drone AI Nexus,19,1851,"431,773,684.20","8,203,699,999.76","533,982,749.42"
6,E-R00025,Skill Injector,3,13,"2,620,000,000.00","7,860,000,000.00","2,403,386,499.64"
7,E-R00028,Sleeper Data Library,29,17207,"257,908,793.10","7,479,355,000.00","166,000,029.37"
8,B-R00007,Ancient Coordinates Database,22,4759,"293,277,272.73","6,452,100,000.00","369,929,750.51"
9,B-R00007,Sleeper Data Library,24,12852,"238,459,583.33","5,723,030,000.00","268,523,589.71"


In [15]:
prices[prices['date'] > days_back(30)] \
    .filter_by(empire['regionID'], 'region_id', exclude=False) \
    .join(regions, on={'region_id': 'regionID'}) \
    .join(types, on={'type_id': 'typeID'}) \
    .groupby(['typeName','regionName'],
       [agg.SUM('volume'),
        agg.AVG('average'),
        agg.STDV('average'),
        agg.AVG('revenue'),
        agg.SUM('revenue'),
#         agg.QUANTILE('avgPrice',[0.25, 0.75])
       ]).topk('Sum of revenue', 50).to_dataframe()

,regionName,typeName,Sum of volume,Avg of average,Stdv of average,Avg of revenue,Sum of revenue
0,The Forge,Skill Injector,207966,"624,515,667.61","6,479,457.80","4,477,096,180,309.36","129,835,789,228,971.33"
1,The Forge,30 Day Pilot's License Extension (PLEX),108826,"1,040,655,800.83","5,803,095.90","3,905,054,053,725.14","113,246,567,558,028.92"
2,The Forge,Skill Extractor,178797,"245,559,309.22","2,690,309.17","1,514,333,873,134.11","43,915,682,320,889.10"
3,Domain,30 Day Pilot's License Extension (PLEX),16085,"1,024,847,604.05","5,104,607.79","568,371,987,604.19","16,482,787,640,521.52"
4,Domain,Skill Injector,20731,"635,867,575.77","7,929,530.64","454,578,220,900.72","13,182,768,406,120.83"
5,The Forge,'Excavator' Mining Drone,5527,"1,437,757,026.33","189,250,708.44","271,448,941,721.50","7,872,019,309,923.49"
6,Sinq Laison,30 Day Pilot's License Extension (PLEX),5810,"1,008,642,454.91","6,987,803.11","202,105,660,324.29","5,861,064,149,404.40"
7,The Forge,Drone Coronary Unit,670269,"6,790,876.32","2,922,570.13","156,749,516,935.77","4,545,735,991,137.25"
8,Sinq Laison,Skill Injector,6780,"646,112,231.05","9,700,859.43","151,059,533,516.15","4,380,726,471,968.46"
9,Lonetrek,23rd Tier Overseer's Personal Effects,32459,"133,837,000.00",0.00,"149,800,523,551.72","4,344,215,183,000.00"


In [16]:
prices[prices['date'] > days_back(30)] \
    .join(regions, on={'region_id': 'regionID'}) \
    .groupby(['regionName', 'region_id'],
       [agg.COUNT(),
        agg.SUM('revenue'),
       ]).topk('Sum of revenue', 100).to_dataframe()

,regionName,region_id,Count,Sum of revenue
0,The Forge,10000002,215692,"571,589,509,446,524.38"
1,Domain,10000043,154926,"76,876,985,283,118.66"
2,Sinq Laison,10000032,118795,"25,575,277,782,371.70"
3,Delve,10000060,53527,"14,579,029,405,974.73"
4,Heimatar,10000030,94868,"13,906,547,422,542.98"
5,Metropolis,10000042,89996,"13,556,248,225,106.54"
6,Lonetrek,10000016,66667,"12,673,797,949,225.18"
7,Essence,10000064,64405,"7,145,788,816,893.95"
8,The Citadel,10000033,56620,"4,623,368,724,334.85"
9,Providence,10000047,50203,"4,125,710,552,805.45"


In [17]:
d = prices[prices['date'] > days_back(10)].filter_by(10000002, 'region_id')
d['average'] = d['average'] / 1e9
d['lowest'] = d['lowest'] / 1e9
d['highest'] = d['highest'] / 1e9
d['revenue'] = d['revenue'] / 1e9

d = d.groupby(['type_id'],
       [agg.COUNT(),
        agg.AVG('average'),
        agg.STDV('average'),
        agg.MIN('lowest'),
        agg.MAX('highest'),
        agg.SUM('volume'),
        agg.SUM('revenue'),
        agg.AVG('revenue'),
        agg.QUANTILE('revenue',[0.25, 0.75]),
        agg.QUANTILE('average',[0.2, 0.8])
       ]).join(types, on={'type_id': 'typeID'})

In [18]:
market_penetration = 0.15
d['Daily margin bln'] = d['Stdv of average'] / d['Avg of average'] * d['Avg of revenue'] * 2
d['Recommended investment'] = d['Quantiles of average'].apply(lambda x: x[0]) * d['Sum of volume'] * market_penetration
d['Potential profit'] = d['Quantiles of average'].apply(lambda x: x[1] - x[0]) * d['Sum of volume'] * market_penetration
d['ROI (volatility)'] = d['Potential profit'] / d['Recommended investment']
d.to_dataframe().sort_values(by='Daily margin bln', ascending=False)[:80]

,type_id,Count,Avg of average,Stdv of average,Min of lowest,Max of highest,Sum of volume,Sum of revenue,Avg of revenue,Quantiles of revenue,Quantiles of average,typeName,Daily margin bln,Recommended investment,Potential profit,ROI (volatility)
5166,40520,9,0.63,0.00,0.62,0.63,63820,"40,049.08","4,449.90","[4051.65224659, 4685.7912406]","[0.623599978, 0.63]",Skill Injector,42.66,"5,969.72",61.27,0.01
6361,28366,9,0.01,0.00,0.01,0.01,213806,"2,037.46",226.38,"[215.802, 258.167113323]","[0.00883801011, 0.01034999997]",Drone Coronary Unit,27.48,283.44,48.49,0.17
3109,40519,9,0.24,0.00,0.24,0.25,56107,"13,679.78","1,519.98","[1392.36566566, 1633.46526487]","[0.24220000001, 0.246001001]",Skill Extractor,22.89,"2,038.37",31.99,0.02
8904,21815,8,0.01,0.00,0.01,0.01,133415,884.05,110.51,"[104.332767723, 127.353954037]","[0.005953, 0.0074]",Elite Drone AI,19.75,119.13,28.96,0.24
2381,29668,9,1.04,0.00,1.03,1.05,33031,"34,263.15","3,807.02","[3600.82869303, 4013.860776]","[1.0345002, 1.0400002]",30 Day Pilot's License Extension (PLEX),19.45,"5,125.59",27.25,0.01
6135,41030,9,1.61,0.04,1.55,1.72,1661,"2,668.64",296.52,"[269.329334642, 332.255017848]","[1.589993999, 1.63229899783]",'Excavator' Mining Drone,13.98,396.15,10.54,0.03
1032,28363,9,0.00,0.00,0.00,0.00,466273,606.61,67.40,"[56.719674, 78.5661111632]","[0.001249, 0.0014220368]",Drone Cerebral Fragment,7.95,87.36,12.10,0.14
610,43681,9,1.61,0.14,1.28,1.80,219,346.75,38.53,"[29.7374519425, 55.8477230688]","[1.44870413581, 1.7988587834]",'Excavator' Ice Harvesting Drone,6.71,47.59,11.50,0.24
5228,25591,9,0.00,0.00,0.00,0.00,2164859,369.38,41.04,"[26.5706288605, 48.009995306]","[0.00015800669, 0.00017500201]",Contaminated Lorentz Fluid,5.01,51.31,5.52,0.11
4631,34,9,0.00,0.00,0.00,0.00,220235314940,931.05,103.45,"[91.5077347835, 123.019559718]","[4.11e-09, 4.32e-09]",Tritanium,4.30,135.78,6.94,0.05


In [19]:
d.save('top200.csv', format='csv')

In [22]:
# PLEX
x = prices[prices['date'] > days_back(20)].join(regions, on={'region_id': 'regionID'}).filter_by(29668, 'type_id').groupby(['regionName'],
       [agg.COUNT(),
        agg.MIN('lowest'),
        agg.MAX('highest'),
        agg.AVG('average'),
        agg.STDV('average'),
        agg.SUM('volume'),
        agg.SUM('revenue'),
        agg.AVG('revenue'),
#         agg.QUANTILE('revenue',[0.25, 0.75]),
        agg.QUANTILE('average',[0.2, 0.8])
       ])
x[x['Avg of average'] > 5e8].topk('Sum of revenue', 100).to_dataframe()

,regionName,Count,Min of lowest,Max of highest,Avg of average,Stdv of average,Sum of volume,Sum of revenue,Avg of revenue,Quantiles of average
0,The Forge,19,"1,026,300,000.00","1,057,349,998.00","1,037,692,163.93","2,975,381.82",69819,"72,439,769,390,098.95","3,812,619,441,584.16","[1034200206.0, 1040500000.0]"
1,Domain,19,"1,016,000,002.01","1,054,968,843.90","1,024,331,439.22","3,482,081.26",10653,"10,908,978,345,453.66","574,156,755,023.88","[1020143330.0, 1027835000.0]"
2,Sinq Laison,19,"999,303,000.00","1,053,551,036.97","1,009,352,369.40","2,693,391.53",3817,"3,852,619,437,989.47","202,769,444,104.71","[1006503000.0, 1010798001.0]"
3,Heimatar,19,"597,800,500.50","1,045,999,999.84","1,021,560,270.62","6,351,029.26",2117,"2,161,207,005,330.44","113,747,737,122.65","[1016675759.7, 1027085899.28]"
4,Metropolis,19,"1,011,010,010.00","1,050,000,000.00","1,023,368,654.16","5,189,892.38",2037,"2,082,974,453,311.05","109,630,234,384.79","[1020000004.0, 1030193470.08]"
5,Lonetrek,19,"1,015,000,001.00","1,069,999,999.99","1,032,290,398.45","5,633,080.48",1033,"1,065,292,952,316.33","56,068,050,121.91","[1027135084.08, 1036213215.03]"
6,Delve,19,"991,444,444.00","1,096,000,000.00","1,027,928,444.05","8,516,467.27",1027,"1,053,650,332,617.48","55,455,280,664.08","[1020018740.28, 1036057356.13]"
7,Essence,19,"1,006,003,000.25","1,084,999,999.00","1,024,003,053.41","6,325,062.81",1016,"1,039,469,480,336.68","54,708,920,017.72","[1019149134.76, 1028913655.54]"
8,Verge Vendor,19,"417,350,000.00","1,094,999,989.96","1,015,026,595.71","9,725,453.51",723,"733,961,624,917.87","38,629,559,206.20","[1012500006.37, 1021211231.41]"
9,The Citadel,19,"1,007,700,500.01","1,089,998,998.00","1,032,151,961.24","8,553,388.98",591,"609,033,566,726.72","32,054,398,248.77","[1023176191.12, 1039903730.4]"


In [23]:
regions[regions['regionName']=='Great Wildlands']

regionID,regionName
10000011,Great Wildlands


In [25]:
# Great Wildlands
prices[prices['date'] > days_back(20)].filter_by(10000011, 'region_id').join(types, on={'type_id': 'typeID'}).groupby(['typeName'],
       [agg.COUNT(),
        agg.AVG('average'),
        agg.STDV('average'),
        agg.SUM('volume'),
        agg.SUM('revenue'),
        agg.AVG('revenue')
       ]).topk('Sum of revenue', 20).to_dataframe()

,typeName,Count,Avg of average,Stdv of average,Sum of volume,Sum of revenue,Avg of revenue
0,Strontium Clathrates,7,"7,712.99","1,276.63",191767,"1,176,935,206.55","168,133,600.94"
1,30 Day Pilot's License Extension (PLEX),1,"1,150,000,000.00",0.00,1,"1,150,000,000.00","1,150,000,000.00"
2,Nitrogen Isotopes,4,999.49,0.88,622844,"621,782,626.68","155,445,656.67"
3,Hydrogen Fuel Block,6,"27,624.33","2,683.40",14160,"388,880,880.00","64,813,480.00"
4,Sleeper Drone AI Nexus,1,"5,000,000.00",0.00,75,"375,000,000.00","375,000,000.00"
5,Ancient Coordinates Database,2,"1,500,000.00",0.00,239,"358,500,000.00","179,250,000.00"
6,Sleeper Data Library,2,"500,000.00",0.00,690,"345,000,000.00","172,500,000.00"
7,Quafe Zero,1,"29,999,998.00",0.00,8,"239,999,984.00","239,999,984.00"
8,Cynosural Field Generator I,18,"2,963,390.09","315,609.63",72,"213,104,443.94","11,839,135.77"
9,Helium Isotopes,6,965.66,74.09,175500,"171,333,500.00","28,555,583.33"


In [ ]:
prices[prices['date'] > days_back(30)].filter_by(
    values=[29668, 40519, 40520],
    column_name='typeID', exclude=False).join(regions, 'regionID').groupby(['regionName'],
       [agg.SUM('orderCount'),
        agg.SUM('revenue'),
        agg.AVG('revenue')
       ]).topk('Sum of revenue', 100).to_dataframe()

In [ ]:
prices[prices['date'] > days_back(20)] \
    .join(types, 'typeID').join(regions, 'regionID') \
    .filter_by(values=['Moros', 'Naglfar', 'Archon', 'Apostle', 'Nidhoggur', 'Chimera', 'Thanatos', 'Phoenix', 'Revelation', 'Rorqual'], column_name='typeName') \
    .groupby(['typeName', 'regionName'],
       [agg.COUNT(),
        agg.MIN('lowPrice'),
        agg.MAX('highPrice'),
        agg.AVG('avgPrice'),
        agg.STDV('avgPrice'),
        agg.SUM('volume'),
        agg.SUM('revenue'),
        agg.AVG('revenue'),
        agg.QUANTILE('avgPrice',[0.2, 0.8])
       ]).topk('Sum of revenue', 100).to_dataframe()

In [ ]:
officers = types[types['typeName'].apply(lambda x: " Modified" in x)]
# removing junk
officers = officers.filter_by(values=(40308, 40312, 363349, 367519, 367548, 367716, 367738, 367742, 367751, 367760, 367885, 367895, 367896, 367897, 367898, 367899, 367900, 368593, 370658),
                   column_name='typeID', exclude=True)
print len(officers)
officers

In [ ]:
#     .filter_by(values=[10000002], column_name='regionID', exclude=True) \
prices[prices['date'] > days_back(150)] \
    .filter_by(values=officers['typeID'], column_name='typeID') \
    .join(types, 'typeID').join(regions, 'regionID') \
    .groupby(['typeName', 'regionName'],
       [agg.COUNT(),
        agg.MIN('lowPrice'),
        agg.MAX('highPrice'),
        agg.AVG('avgPrice'),
        agg.STDV('avgPrice'),
        agg.SUM('volume'),
        agg.SUM('revenue'),
        agg.AVG('revenue'),
        agg.QUANTILE('avgPrice',[0.2, 0.8])
       ]).to_dataframe().sort_values(by='Stdv of avgPrice', ascending=False)

In [ ]:
drone_officers = officers[officers.apply(lambda x: 'Unit ' in x['typeName'])]

In [26]:
ts_forge = prices.filter_by(10000002, 'region_id').join(types, {'type_id':'typeID'}).to_dataframe().set_index('date')

In [ ]:
ts_forge[ts_forge.typeID.isin(drone_officers['typeID'])].sort_index()['2016-10':]

In [ ]:
import seaborn as sns

citadel_materials = (
    'Integrity Response Drones',
    'Organic Mortar Applicators',
    'Broadcast Node',
    'Nano-Factory',
    'Recursive Computing Module',
    'Self-Harmonizing Power Core',
    'Sterile Conduits',
    'Wetware Mainframe'
)
officers = (
    "Gotan's Modified Gyrostabilizer",
    "Gotan's Modified Heavy Warp Disruptor" 
# "Chelm's Modified Heat Sink", # 6.1 bils
# "Chelm's Modified Power Diagnostic System", # 5.2 bils 
# "Chelm's Modified Energized Kinetic Membrane", # 1.7 bils
# "Chelm's Modified Mega Beam Laser", # 1 bil
# "Chelm's Modified Large Armor Repairer", # 800 mils
# "Unit W-634's Modified Drone Damage Amplifier"
)

officers_DCU = types[types['typeName'].apply(lambda x: " Modified Damage Control" in x)]['typeName']

rmt_tokens = ("30 Day Pilot's License Extension (PLEX)", "Skill Extractor", "Skill Injector")


for i in officers_DCU:
    ts_forge[ts_forge.typeName == i].average.plot(figsize=(16, 12), label=i, logy=False)

plt.legend(loc='best');

In [ ]:
unit634 = types[types['typeName'].apply(lambda x: "Unit W-634's Modified" in x)]['typeName']

for i in unit634:
    series = ts_forge[ts_forge.typeName == i]
    if not series.empty:
        series.average.plot(figsize=(16, 12), label=i, logy=False)

plt.legend(loc='best');

In [ ]:
officer_rhml = types[types['typeName'].apply(lambda x: 'Modified Rapid Heavy Missile Launcher' in x)]['typeName']

for i in officer_rhml:
    series = ts_forge[ts_forge['typeName'] == i]
    if not series.empty:
        series.average.plot(figsize=(16, 12), label=i, logy=False)

plt.legend(loc='best');
plt.show();

# for i in officer_rhml[5:]:
#     series = ts_forge[ts_forge['typeName'] == i]
#     if not series.empty:
#         series.average.plot(figsize=(16, 12), label=i, logy=False)

# plt.legend(loc='best');
# plt.show();

In [ ]:
officer_rhml

In [ ]:
prices[prices['date'] > days_back(50)] \
    .join(types, 'typeID').join(regions, 'regionID') \
    .filter_by(values=officers['typeName'], column_name='typeName').topk(column_name='highPrice', k=20).to_dataframe()

In [ ]:
items_dict = {v.decode('utf-8'): k for k, v in items.set_index('typeID').typeName.dropna().to_dict().iteritems()}
len(items_dict)

In [ ]:
def esi_market_hist(typeid, regionid=10000002):
    url = 'https://esi.tech.ccp.is/latest/markets/{}/history/?type_id={}&datasource=tranquility'.format(regionid, typeid)
    return pd.read_json(url).set_index('date')

In [ ]:
esi_market_hist(items_dict["Draclira's Modified Capacitor Power Relay"]).avgPrice.plot(figsize=(15,10))

In [ ]:
esi_market_hist(29668)['2016-07':][['highest','lowest','average']].plot(figsize=(15,10)).set_title('Plex')

In [ ]:
data = esi_market_hist(items_dict['Skill Injector'])['2016-07':]
ma = data.avgPrice.rolling(20).mean()
mstd = data.avgPrice.rolling(20).std()
moving_min = data.lowPrice.rolling(5).min()
moving_max = data.highPrice.rolling(5).max()

plt.figure(figsize=(15,10))

plt.plot(data.index, data.avgPrice, '--ko')

plt.plot(ma.index, ma, 'b', label='MA 20d')

# plt.fill_between(mstd.index, ma-2*mstd, ma+2*mstd, color='b', alpha=0.2, label='MSTD')
plt.fill_between(moving_max.index, moving_min, moving_max, color='c', alpha=0.3, label='Donchian')
plt.fill_between(data.index, data.highPrice, data.lowPrice, color='r', alpha=0.5, label='OHLC')
plt.legend(loc='best');

In [ ]:
esi_market_hist(35825).plot(figsize=(15,10)).set_title('Raitaru')